In [3]:
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter('ignore')

In [8]:
# размер тестовой выборки
test_size=945
# указать путь до исходного файла с данными
df_ivs = pd.read_excel('data/Protva_3.xlsx')
# выбираются переменные-предикторы из изходного файла 
qt_cols = [col for col in df_ivs.columns[1:] if 'Q(t+' not in col]
len(qt_cols)
X = df_ivs.loc[:, qt_cols]

# предикат
ycols = [col for col in df_ivs.columns[1:] if 'Q(t+' in col][::-1]

y = df_ivs.loc[:, ycols] #.iloc[:, -1]
y_train = y[test_size:].reset_index(drop=True)
y_test = y[:test_size].reset_index(drop=True)

X_train = X[test_size:]
X_test = X[:test_size]
print('Train:', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train: (3781, 57) (3781, 7)
Test:  (945, 57) (945, 7)


In [55]:
corr = round(df_ivs.corr(), 2)
rownames = [k for k in corr.columns if 't+' in k]
colnames = [k for k in corr.columns if 't+' not in k]
print(len(rownames), colnames)
corr.to_excel('results/Protva_corr.xlsx')
corr.loc[rownames, colnames].style.background_gradient(cmap='coolwarm')

7 ['QtSZ', 'Q(t-1)SZ', 'Q(t-2)SZ', 'Q(t-3)SZ', 'Q(t-4)SZ', 'Q(t-5)SZ', 'Q(t-6)SZ', 'Q(t-7)SZ', 'Δqmax2', 'Δqmax3', 'Δqmax4', 'Δqmax5', 'Δqmax6', 'Δqmax7', '27509(t)', '27509(t-1)', '27509(t-2)', '27509(t-3)', '27509(t-4)', '27509(t-5)', '27509(t-6)', '27509(t-7)', '27606(t)', '27606(t-1)', '27606(t-2)', '27606(t-3)', '27606(t-4)', '27606(t-5)', '27606(t-6)', '27606(t-7)', '27611(t)', '27611(t-1)', '27611(t-2)', '27611(t-3)', '27611(t-4)', '27611(t-5)', '27611(t-6)', '27611(t-7)', 'P(t)avg', 'P(t-1)avg', 'P(t-2)avg', 'P(t-3)avg', 'P(t-4)avg', 'P(t-5)avg', 'P(t-6)avg', 'P(t-7)avg', 'sum2', 'sum3', 'sum4', 'sum5', 'sum6', 'sum7', 'Iw', 'T>0', 'T>2', 'T>5', 'Evap']


,QtSZ,Q(t-1)SZ,Q(t-2)SZ,Q(t-3)SZ,Q(t-4)SZ,Q(t-5)SZ,Q(t-6)SZ,Q(t-7)SZ,Δqmax2,Δqmax3,Δqmax4,Δqmax5,Δqmax6,Δqmax7,27509(t),27509(t-1),27509(t-2),27509(t-3),27509(t-4),27509(t-5),27509(t-6),27509(t-7),27606(t),27606(t-1),27606(t-2),27606(t-3),27606(t-4),27606(t-5),27606(t-6),27606(t-7),27611(t),27611(t-1),27611(t-2),27611(t-3),27611(t-4),27611(t-5),27611(t-6),27611(t-7),P(t)avg,P(t-1)avg,P(t-2)avg,P(t-3)avg,P(t-4)avg,P(t-5)avg,P(t-6)avg,P(t-7)avg,sum2,sum3,sum4,sum5,sum6,sum7,Iw,T>0,T>2,T>5,Evap
Q(t+7)SZ,0.45,0.39,0.33,0.28,0.24,0.21,0.17,0.15,0.34,0.36,0.37,0.37,0.37,0.37,0.02,0.02,0.01,0.01,0,-0,-0,-0,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.02,-0.01,-0.16,-0.16,-0.16,-0.09
Q(t+6)SZ,0.53,0.45,0.39,0.33,0.28,0.24,0.21,0.17,0.4,0.42,0.43,0.43,0.43,0.42,0.03,0.02,0.02,0.01,0.01,0,-0,-0,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.03,0.03,0.03,0.03,0.03,-0.01,-0.16,-0.16,-0.16,-0.08
Q(t+5)SZ,0.61,0.53,0.45,0.39,0.33,0.28,0.24,0.21,0.46,0.49,0.5,0.5,0.49,0.49,0.03,0.03,0.02,0.02,0.01,0.01,0,-0,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.03,0.03,0.03,0.03,0.03,0.03,-0,-0.16,-0.16,-0.16,-0.07
Q(t+4)SZ,0.7,0.61,0.53,0.45,0.39,0.33,0.28,0.24,0.53,0.56,0.58,0.58,0.57,0.56,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.03,0.04,0.04,0.04,0.04,0.04,0,-0.16,-0.16,-0.16,-0.06
Q(t+3)SZ,0.8,0.7,0.61,0.53,0.45,0.39,0.33,0.28,0.59,0.65,0.67,0.67,0.66,0.65,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.03,0.03,0.02,0.02,0.02,0.01,0.01,0.01,0.03,0.04,0.04,0.04,0.04,0.04,0.01,-0.16,-0.16,-0.16,-0.05
Q(t+2)SZ,0.9,0.8,0.7,0.61,0.53,0.45,0.39,0.33,0.62,0.7,0.74,0.75,0.74,0.73,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.03,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.03,0.03,0.03,0.02,0.02,0.02,0.01,0.01,0.03,0.04,0.04,0.05,0.05,0.05,0.01,-0.16,-0.16,-0.16,-0.04
Q(t+1)SZ,0.97,0.9,0.8,0.7,0.61,0.53,0.45,0.39,0.58,0.72,0.78,0.8,0.81,0.8,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.03,0.03,0.02,0.02,0.02,0.02,0.01,0.01,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.01,0.04,0.04,0.05,0.05,0.05,0.05,0.02,-0.16,-0.16,-0.16,-0.03


In [9]:
params_lm= \
{
    'fit_intercept' : True,
    'normalize': False
    }
# инициализируем модель с параметрами
lm = MultiOutputRegressor(LinearRegression(**params_lm), n_jobs=7)
# обучаем модель
lm.fit(X_train, y_train)

MultiOutputRegressor(estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                                n_jobs=None, normalize=False),
                     n_jobs=7)

In [14]:
# хронологическая модель
df_chrono = df_ivs.sort_values(['date']).reset_index(drop=True)
# print(df_chrono.head())

# переменные-предикторы остаются те же
X_chrono = df_chrono.loc[:, qt_cols]
# как и предикат
y_chrono = df_chrono.loc[:, ycols] #.iloc[:, -1]
# и размер тестовой выборки
y_chrono_train = y_chrono[test_size:].reset_index(drop=True)
y_chrono_test = y_chrono[:test_size].reset_index(drop=True)

X_chrono_train = X_chrono[test_size:]
X_chrono_test = X_chrono[:test_size]
print('Train:', X_chrono_train.shape, y_chrono_train.shape)
print('Test: ', X_chrono_test.shape, y_chrono_test.shape)

        date  Q(t+7)SZ  Q(t+6)SZ  Q(t+5)SZ  Q(t+4)SZ  Q(t+3)SZ  Q(t+2)SZ  \
0 2003-01-07      8.39      8.23      8.39      8.23      8.56      8.07   
1 2003-01-08      8.23      8.39      8.23      8.39      8.23      8.56   
2 2003-01-09      8.07      8.23      8.39      8.23      8.39      8.23   
3 2003-01-10      8.23      8.07      8.23      8.39      8.23      8.39   
4 2003-01-11      8.07      8.23      8.07      8.23      8.39      8.23   

   Q(t+1)SZ  QtSZ  Q(t-1)SZ  ...      sum3      sum4       sum5       sum6  \
0      8.23  9.20      9.37  ...  3.166667  3.833333  13.600000  14.566667   
1      8.07  8.23      9.20  ...  0.200000  3.166667   3.833333  13.600000   
2      8.56  8.07      8.23  ...  0.466667  0.666667   3.633333   4.300000   
3      8.23  8.56      8.07  ...  1.866667  1.866667   2.066667   5.033333   
4      8.39  8.23      8.56  ...  1.866667  1.866667   1.866667   2.066667   

        sum7   Iw  T>0  T>2  T>5  Evap  
0  16.066667  0.0  0.0  0.0  0.0 

In [15]:
# обучаем хронологическую модель с теми же параметрами
lm_chrono = MultiOutputRegressor(LinearRegression(**params_lm), n_jobs=7)
# обучаем модель
lm_chrono.fit(X_chrono_train, y_chrono_train)

MultiOutputRegressor(estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                                n_jobs=None, normalize=False),
                     n_jobs=7)

In [ ]:
# тестируем все вместе
pred = pd.DataFrame(columns=['metric'] + ycols)
train_pred = pd.DataFrame(lm.predict(X_train))
train_pred_chrono = pd.DataFrame(lm_chrono.predict(X_chrono_train))
test_pred = pd.DataFrame(lm.predict(X_test))
test_pred_chrono = pd.DataFrame(lm_chrono.predict(X_chrono_test))
for i, y in enumerate(ycols):
    pred.at[1,'metric'] = 'RMSEtrain'
    pred.at[1,y] = RMSE(y_train.loc[:,y], train_pred.loc[:,i])
    pred.at[2,'metric'] = 'RMSEtrain_chrono'
    pred.at[2,y] = RMSE(y_chrono_train.loc[:,y], train_pred_chrono.loc[:,i])
    
    pred.at[3,'metric'] = 'NSEtrain'
    pred.at[3,y] = NSE(y_train.loc[:,y], train_pred.loc[:,i])
    pred.at[4,'metric'] = 'NSEtrain_chrono'
    pred.at[4,y] = NSE(y_chrono_train.loc[:,y], train_pred_chrono.loc[:,i])
    
    pred.at[5,'metric'] = 'RMSEtest'
    pred.at[5,y] = RMSE(y_test.loc[:,y], test_pred.loc[:,i])
    pred.at[6,'metric'] = 'RMSEtest_chrono'
    pred.at[6,y] = RMSE(y_chrono_test.loc[:,y], test_pred_chrono.loc[:,i])
    
    pred.at[7,'metric'] = 'NSEtest'
    pred.at[7,y] = NSE(y_test.loc[:,y], test_pred.loc[:,i])
    pred.at[8,'metric'] = 'NSEtest_chrono'
    pred.at[8,y] = NSE(y_chrono_test.loc[:,y], test_pred_chrono.loc[:,i])
pred.to_excel('results/protva_lm_refernce+chrono.xlsx')
pred = pred.T
pred.columns = pred.iloc[0,:]
pred = pred.drop(['metric'])
pred.plot(kind='bar', subplots='True', figsize=[16,16])
plt.show()
pred

In [53]:
df_test = pd.concat([test_pred, y_test, test_pred_chrono, y_chrono_test], axis=1)
df_test.to_excel('results/protva_all_year_lm_reference_test_chrono.xlsx')
df_test.head()

,0,1,2,3,4,5,6,Q(t+1)SZ,Q(t+2)SZ,Q(t+3)SZ,...,4,5,6,Q(t+1)SZ,Q(t+2)SZ,Q(t+3)SZ,Q(t+4)SZ,Q(t+5)SZ,Q(t+6)SZ,Q(t+7)SZ
0,22.588402,24.670478,26.070125,26.695368,27.454732,26.877788,26.185721,19.50,18.3,18.5,...,15.648423,17.634366,19.459152,8.23,8.07,8.56,8.23,8.39,8.23,8.39
1,13.647222,14.466654,15.334664,16.370189,17.682209,18.936323,19.952481,12.80,13.0,13.2,...,14.354506,16.276990,18.193746,8.07,8.56,8.23,8.39,8.23,8.39,8.23
2,12.999319,14.114982,15.422740,17.005930,18.703294,20.312035,21.884578,12.20,12.8,12.6,...,14.806053,16.862497,18.847524,8.56,8.23,8.39,8.23,8.39,8.23,8.07
3,10.361589,11.715340,13.401097,15.266197,17.146371,18.856857,20.605744,9.66,10.1,10.5,...,16.058300,18.049993,19.845540,8.23,8.39,8.23,8.39,8.23,8.07,8.23
4,11.909378,12.203734,12.550376,12.909780,13.041814,13.235871,13.498278,11.70,11.5,10.9,...,15.026995,17.106077,19.060856,8.39,8.23,8.39,8.23,8.07,8.23,8.07


In [54]:
df_train = pd.concat([train_pred, y_train, train_pred_chrono, y_chrono_train], axis=1)
df_train.to_excel('results/protva_all_year_lm_reference_train_chrono.xlsx')
df_train.head()

,0,1,2,3,4,5,6,Q(t+1)SZ,Q(t+2)SZ,Q(t+3)SZ,...,4,5,6,Q(t+1)SZ,Q(t+2)SZ,Q(t+3)SZ,Q(t+4)SZ,Q(t+5)SZ,Q(t+6)SZ,Q(t+7)SZ
0,10.535263,11.390501,12.374084,13.515880,14.351239,14.425712,14.860115,9.92,11.00,11.60,...,13.370564,13.394475,13.273813,10.6,10.6,10.6,10.4,10.3,10.1,10.1
1,11.840711,12.422306,13.417040,14.385948,15.458449,16.939919,18.172389,11.70,11.40,11.40,...,13.337309,13.447867,13.369985,10.6,10.6,10.4,10.3,10.1,10.1,10.1
2,11.218838,11.579167,11.641733,11.597733,11.565615,11.421756,11.401922,10.80,11.70,13.50,...,13.482762,13.784170,13.768804,10.6,10.4,10.3,10.1,10.1,10.1,10.1
3,8.150998,8.978448,10.015837,11.570561,12.253152,13.263657,13.490400,7.87,8.04,8.00,...,12.355438,12.361751,12.255506,10.4,10.3,10.1,10.1,10.1,10.1,10.2
4,9.791304,10.196929,10.631094,11.205093,11.853053,12.471769,13.123277,9.27,9.27,9.27,...,11.081155,11.214675,11.359936,10.3,10.1,10.1,10.1,10.1,10.2,10.2
